In [ ]:

%load_ext autoreload
%autoreload 2

import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

# from dynamicslib.common import f_
from dynamicslib.consts import muEM
from dynamicslib.continuation import f_df_CR3_single, find_bif, arclen_cont, arclen_to_fail
from dynamicslib.common import prop_ic, get_JC_tf, eom
from dynamicslib.integrator import dop853
from tqdm.auto import tqdm
# from dynamicslib.targeter import *
from dynamicslib.plotting import plotly_display

file = "path_here/L2 Lyapunov.csv"
db = pd.read_csv(file).set_index("Index")

# TODO: L3 things
# TODO: L1 halo tangent at 46->47 (maybe a L4/5 vertical?), 70, 92, everything after 93 (check hopfs and tangents)
# TODO: L2 halo, tangent at 208, tangent at 470 ish
# TODO: L2 Lyapunov tangent(prolly just axial), second quad, triple (add more of these)
# TODO: L4 axial, DRO axial? Resonances
# TODO: halo tangents
# TODO: get the L4 verts further
# TODO: late halo bifs
# TODO: period 5, 6
# TODO: Lissajous
# TODO: Poisson
# TODO: Homotopy
# TODO: transfers?
# TODO: 4 and 3 butterfly opposite directions?

In [ ]:
def xtf2X(x0, tf):
    return np.array([x0[0], x0[2], x0[-2], tf / 2])


def X2xtf(X):
    return np.array([X[0], 0, X[1], 0, X[2], 0]), X[-1] * 2


def stmeom2DF(eomf, stm):
    dF = np.array(
        [
            [stm[1, 0], stm[1, 2], stm[1, -2], eomf[1]],
            [stm[-3, 0], stm[-3, 2], stm[-3, -2], eomf[-3]],
            [stm[-1, 0], stm[-1, 2], stm[-1, -2], eomf[-1]],
        ]
    )
    return dF


def f_func(x0, tf, xf):
    return np.array([xf[1], xf[-3], xf[-1]])


func = lambda X: f_df_CR3_single(X, X2xtf, stmeom2DF, f_func, False, muEM, 1e-10)

In [ ]:
if "Initial z" not in db.columns:
    db["Initial z"] = 0
X0 = db[["Initial x", "Initial z", "Initial vy", "Period"]].iloc[755].values
Xm1 = db[["Initial x", "Initial z", "Initial vy", "Period"]].iloc[754].values
X0[-1] /= 2
Xm1[-1] /= 2

In [ ]:
X0, _ = find_bif(
    Xm1,
    func,
    X0 - Xm1,
    2e-3,
    targ_tol=1e-12,
    bisect_tol=1e-11,
    bif_type="period triple",
    debug=False,
)

In [ ]:
# switch direction!
x0, tf = X2xtf(X0)
ts, xs, _ = dop853(eom, (0, tf / 2), x0, 1e-14, 0)
xs[1, -1] = 0
X0 = xtf2X(xs[:, -1], ts[-1] * 2)

In [ ]:
X1 = X0.copy()
X1[-1] *= 3
f, df, stm = func(X1)
svd = np.linalg.svd(df)
tangent2 = svd.Vh[-2]

In [ ]:
Xsp1, esp1 = arclen_cont(X1, func, -tangent2, 5e-5, 1e-3, 1e-9, exact_tangent=True,max_iter=30)

In [ ]:
# Xsm1, esm1 = arclen_cont(X1, func, -tangent2, 1e-3, 0.1, 1e-10, use_exact_tangent=True)

In [ ]:
Xsp2, esp2 = arclen_cont(Xsp1[-1], func, Xsp1[-1] - Xsp1[-2], 1e-4, 2.5e-2, 1e-9, max_iter=50)

In [ ]:
Xsp3, esp3 = arclen_cont(Xsp2[-1], func, Xsp2[-1] - Xsp2[-2], 1e-4, 5e-2, 1e-8, max_iter=50)

In [ ]:
Xsp4, esp4 = arclen_cont(Xsp3[-1], func, Xsp3[-1] - Xsp3[-2], 1.2e-4, 0.05, 1e-8, max_iter=50)

In [ ]:
Xsp5, esp5 = arclen_cont(Xsp4[-1], func, Xsp4[-1] - Xsp4[-2], 1.3e-4, 0.1, 1e-8, max_iter=50)

In [ ]:
Xsp6, esp6 = arclen_cont(Xsp5[-1], func, Xsp5[-1] - Xsp5[-2], 1.4e-4, 0.2, 5e-8, max_iter=50)

In [ ]:
Xsp7, esp7 = arclen_cont(Xsp6[-1], func, Xsp6[-1] - Xsp6[-2], 3e-3, 3, 1e-8, max_iter=50)

In [ ]:
Xsp8, esp8 = arclen_cont(Xsp7[-1], func, Xsp7[-1] - Xsp7[-2], 1e-2, 7, 1e-8, max_iter=50)

In [ ]:
# Xsp7, esp7 = arclen_cont(Xsp6[-1], func, Xsp6[-1] - Xsp6[-2], 1e-2, 1.6, 1e-8, max_iter=50)

In [ ]:
# Xsp8, esp8 = arclen_cont(Xsp7[-1], func, Xsp7[-1] - Xsp7[-2], 8e-3, 2, 1e-8, max_iter=50)

In [ ]:
# Xsp9, esp9 = arclen_cont(Xsp8[-1], func, Xsp8[-1] - Xsp8[-2], 3e-3, 0.5, 1e-8, max_iter=50)

In [ ]:
# _, df, _ = func(Xsp2[-1])
# tangent = np.linalg.svd(df).Vh[-1]

# Xsp3, esp3 = arclen_to_fail(Xsp2[-1], func, tangent, 5e-3, 1e-8, 30)

In [ ]:
Xsm = []
esm = []
Xsp = [*Xsp1, *Xsp2, *Xsp3, *Xsp4, *Xsp5]
esp = [*esp1, *esp2, *esp3, *esp4, *esp5]

param_names = [
    "Index",
    "Initial x",
    "Initial z",
    "Initial vy",
    "Period",
    "Jacobi Constant",
    "Stability Index",
    "Eig1",
    "Eig2",
    "Eig3",
    "Eig4",
    "Eig5",
    "Eig6",
]

xyzs = []
data = []


iterator = list(enumerate(zip(Xsm, esm)))[::-1]
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic(Xcp, X2xtf))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    data.append([-i, *Xcp[:-1], period, jc, stabind, *evals])

iterator = list(enumerate(zip(Xsp, esp)))
for i, (X, evals) in tqdm(iterator):
    Xcp = X.copy()
    xyzs.append(prop_ic(Xcp, X2xtf))
    args = np.argsort(np.abs(evals))
    stabind = np.max([(np.abs(lam) + 1 / np.abs(lam)) / 2 for lam in evals])
    jc, period = get_JC_tf(Xcp, X2xtf)
    data.append([i, *Xcp[:-1], period, jc, stabind, *evals])

In [ ]:
df = pd.DataFrame(data, columns=param_names).set_index("Index")
df.to_csv("L2 Lyapunov P3b (spatial).csv")

In [ ]:
a = None
b = None
c = 10
plotly_display(xyzs[a:b:c], df[a:b:c])